In [48]:
import torch
import torch.nn as nn
import torch.optim as optim
import os
import torch
from torch.utils.data import Dataset, DataLoader
import glob
import matplotlib.pyplot as plt
import time
from datetime import datetime
from SAEModel import SparseAutoencoder
from torch.nn import functional as F
import json

input_dim = 3072  # Input and output dimensions
hidden_dim = 3072*10  # Hidden layer dimension
model = SparseAutoencoder(input_dim, hidden_dim, k = 128, dead_steps_threshold=1000000) # initial lambda is 0

In [49]:
# state_dict = torch.load("./experiments/K_128_20241202/models/model_epoch_40.pt")
state_dict = torch.load("./models_working_1/model_epoch_34.pt")
# Load the weights into the model
model.load_state_dict(state_dict)

<All keys matched successfully>

In [50]:
model

SparseAutoencoder()

## Modified Dataset to also return text with embedding for ease of visualisation

In [51]:

class EmbeddingDataset(Dataset):
    def __init__(self, path, file_pattern, use_files = 40):
        # Load all `.pt` files based on the pattern
        self.file_path = path
        self.files = sorted(glob.glob(path+"/"+file_pattern))
        print(f"Num of files found : {len(self.files)}")
        print(f"Num of files used : {len(self.files[:use_files])}")
        self.data = []

        # Read and store all embeddings from all files
        for file in self.files[:use_files]:
            batch_data = torch.load(file)
            # Extract embeddings and flatten them into a list
            self.data.extend([(item["embedding"], item["text"]) for item in batch_data])

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

### Checking with 2 books data, to see if their dictionaries capture their main idea

In [33]:
path = "./dataset/testing"
file_pattern = "p*.pt"  # Adjust the path if needed
prideAndPred = EmbeddingDataset(path, file_pattern , use_files=20)

path = "./dataset/testing"
file_pattern = "t*.pt"  # Adjust the path if needed
tomSawyer = EmbeddingDataset(path, file_pattern , use_files=20)


prideAndPred_loader = DataLoader(prideAndPred, batch_size=1024, shuffle=True)
tomSawyer_loader = DataLoader(tomSawyer, batch_size=1024, shuffle=True)

Num of files found : 1
Num of files used : 1
Num of files found : 1
Num of files used : 1


(tensor([ 1.5156, -0.8906, -0.3945,  ..., -0.3770, -0.1543,  1.9844],
        dtype=torch.bfloat16),
 'In')

In [ ]:
def getDict(loader):
    word_active_neurons = {}
    with torch.no_grad():
        for batch_num, batch in tqdm(enumerate(loader), total = len(loader), desc="Processing Batches"):
            # Batch to GPU
            batch, text = batch
            batch = batch.cuda().to(torch.float32)

            # Perform forward pass to get latent representations
            recons, auxk, num_dead, latents = model(batch)  # Assuming `model` returns latent as the second output

            # Find top K active neuron indices for each input in the batch
            K = 10  # Assuming TopK SAE is configured with K active neurons
            active_neuron_indices = torch.topk(latents, 128, dim=1).indices

            # Save results
            for i, indices in enumerate(active_neuron_indices):
                word = text[i]  # Assuming dataset has words or identifiers
                word_active_neurons[word] = indices.cpu().tolist()

            if batch_num % 100 == 0:
                print(f"Processed Batch {batch_num}...")
    return word_active_neurons

# # Save the mapping to a file
# with open("word_active_neurons.json", "w") as f:
#     json.dump(word_active_neurons, f)

# print("Forward pass completed and saved!")

In [ ]:
tomSawyerDict = getDict(tomSawyer_loader)

In [ ]:
prideAndPredDict = getDict(prideAndPred_loader)

In [ ]:
from collections import Counter
def most_common_neurons(neuron_dict, K):
    # Flatten all indices into a single list
    all_indices = [index for indices in neuron_dict.values() for index in indices]
    
    # Count the frequency of each index
    index_counts = Counter(all_indices)
    
    # Sort by frequency (descending) and then by index (ascending)
    sorted_indices = sorted(index_counts.items(), key=lambda x: (-x[1], x[0]))
    
    # Return the top K indices
    return [index for index, _ in sorted_indices[:K]]

In [ ]:
def get_word_list(index):
    word_list = []
    for key, val in word_active_neurons.items():
        if index in val:
            word_list.append(key)
    return word_list


In [ ]:
print(get_word_list(18326))

In [5]:
# path = "./dataset/"
# file_pattern = "residual_data_batch_*.pt"  # Adjust the path if needed
# embedding_dataset = EmbeddingDataset(path, file_pattern , use_files=20)

Num of files found : 65
Num of files used : 20


In [6]:
# train_loader = DataLoader(embedding_dataset, batch_size=1024, shuffle=True)

In [53]:
model.eval()

SparseAutoencoder()

In [54]:
model = model.cuda()

In [55]:
from tqdm import tqdm

In [56]:
# word_active_neurons = {}
# with torch.no_grad():
#     for batch_num, batch in tqdm(enumerate(train_loader), total = len(train_loader), desc="Processing Batches"):
#         # Batch to GPU
#         batch, text = batch
#         batch = batch.cuda().to(torch.float32)
        
#         # Perform forward pass to get latent representations
#         recons, auxk, num_dead, latents = model(batch)  # Assuming `model` returns latent as the second output
        
#         # Find top K active neuron indices for each input in the batch
#         K = 10  # Assuming TopK SAE is configured with K active neurons
#         active_neuron_indices = torch.topk(latents, 128, dim=1).indices
        
#         # Save results
#         for i, indices in enumerate(active_neuron_indices):
#             word = text[i]  # Assuming dataset has words or identifiers
#             word_active_neurons[word] = indices.cpu().tolist()
            
#         if batch_num % 100 == 0:
#             print(f"Processed Batch {batch_num}...")

# # # Save the mapping to a file
# # with open("word_active_neurons.json", "w") as f:
# #     json.dump(word_active_neurons, f)

# # print("Forward pass completed and saved!")

Processing Batches:   0%|          | 2/1086 [00:00<04:08,  4.36it/s]

Processed Batch 0...


Processing Batches:   9%|▉         | 102/1086 [00:18<02:56,  5.59it/s]

Processed Batch 100...


Processing Batches:  19%|█▊        | 202/1086 [00:36<02:41,  5.48it/s]

...

Processing Batches:  83%|████████▎ | 902/1086 [02:46<00:34,  5.32it/s]

Processed Batch 900...


Processing Batches:  92%|█████████▏| 1002/1086 [03:05<00:15,  5.43it/s]

Processed Batch 1000...


Processing Batches: 100%|██████████| 1086/1086 [03:20<00:00,  5.42it/s]


In [13]:
with open("word_active_neurons_2Dec.json", "w") as f:
    json.dump(word_active_neurons, f)

In [14]:
torch.save(word_active_neurons, f"./output_2Dec.pt")

In [15]:
import csv
csv_file = './experiments/K_128_20241202/word_active_neurons.csv'

# Open the CSV file for writing
with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)
    
    # Write the header (optional, for clarity)
    header = ['word'] + [f'num{i+1}' for i in range(128)]  # Generate column names like num1, num2, ...
    writer.writerow(header)
    
    # Write each word and its corresponding list of 128 numbers
    for word, numbers in word_active_neurons.items():
        writer.writerow([word] + numbers)

print(f"Data successfully saved to {csv_file}")

Data successfully saved to ./experiments/K_128_20241202/word_active_neurons.csv


In [ ]:
def get_word_list(index):
    word_list = []
    for key, val in word_active_neurons.items():
        if index in val:
            word_list.append(key)
    return word_list


In [57]:
print(get_word_list(0)) #Names?

['eth', 'King', 'Andy', 'ey', 'Fin', 'ack', 'dy', 'ilda', 'Elizabeth', 'Guy', 'gers', 'speaker', 'Lion', 'Rome', 'Mario', 'prince', 'icus', 'host', 'Castro', 'assen', 'Jack', 'iver', 'Martin', 'ome', 'Spain', 'Rose', 'Han', 'Susan', 'Napoleon', 'alf', 'rah', 'enger', 'vie', 'Wolf', 'Ron', 'Haz', 'ku', 'Harris', 'Matrix', 'uzz', 'Pi', 'Beth', 'Lincoln', 'etta', 'ione', 'Duke', 'Ruth', 'Bull', 'Ash', 'RED', 'Robinson', 'iki', 'olas', 'ierre', 'obe', 'ala', 'ocket', 'Lan', 'oco', 'Russell', 'Jesus', 'gang', 'nik', 'Arag', 'boys', 'edi', 'iley', 'Wes', 'Herbert', 'Neil', 'ario', 'Nick', 'nier', 'XVI', 'Ministry', 'etto', 'hof', 'Farm', 'anos', 'Elli', 'Bruce', 'Committee', 'Luigi', 'Bayern']


In [58]:
print(get_word_list(1)) # Verbs?

['cludes', 'oses', 'decided', 'commit', 'figured', 'promised', 'determined', 'ered', 'agreed', 'decide', 'wanted', 'cide', 'dedicated', 'managed', 'chose', 'convinced', 'arks', 'concluded', 'attempted', 'switched', 'decid', 'Figure', 'refuse']


In [59]:
print(get_word_list(2))

['King', 'al', 'Cris', 'Swift', 'dy', 'Lee', 'ilda', 'ingo', 'Hamilton', 'itch', 'Korea', 'idas', 'iden', 'boards', 'BI', 'rost', 'ier', 'Jul', 'icus', 'Ferdinand', 'ko', 'iet', 'Taylor', 'ads', 'assen', 'bage', 'ER', 'Kelly', 'Night', 'Germ', 'Wolf', 'inci', 'hart', 'ource', 'Pi', 'etta', 'Adam', 'Duke', 'horses', 'Bull', 'Robinson', 'anto', 'olan', 'ierre', 'inek', 'Moore', 'PA', 'Lucas', 'Budapest', 'ima', 'ocket', 'ange', 'Gilbert', 'chev', 'oku', 'fold', 'ga', 'nik', 'Factory', 'jin', 'iley', 'elin', 'Alert', 'Columb', 'baum', 'urst', 'ensen', 'owski', 'uffer', 'boa', 'hof', 'anos', 'Inn', 'eny', 'adi', 'ritz', 'Pitts']


In [60]:
print(get_word_list(4))  # Rapid and instantly

['benefits', 'rising', 'rapidly', 'quick', 'faster', 'massive', 'entially', 'evol', 'gent', 'atic', 'Rap', 'sudden', 'ancement', 'rapid', 'mediate', 'exponential', 'racing', 'rá']


In [61]:
for i in [2,4,5,10,15,21]:
    print(get_word_list(i))
    print()

['King', 'al', 'Cris', 'Swift', 'dy', 'Lee', 'ilda', 'ingo', 'Hamilton', 'itch', 'Korea', 'idas', 'iden', 'boards', 'BI', 'rost', 'ier', 'Jul', 'icus', 'Ferdinand', 'ko', 'iet', 'Taylor', 'ads', 'assen', 'bage', 'ER', 'Kelly', 'Night', 'Germ', 'Wolf', 'inci', 'hart', 'ource', 'Pi', 'etta', 'Adam', 'Duke', 'horses', 'Bull', 'Robinson', 'anto', 'olan', 'ierre', 'inek', 'Moore', 'PA', 'Lucas', 'Budapest', 'ima', 'ocket', 'ange', 'Gilbert', 'chev', 'oku', 'fold', 'ga', 'nik', 'Factory', 'jin', 'iley', 'elin', 'Alert', 'Columb', 'baum', 'urst', 'ensen', 'owski', 'uffer', 'boa', 'hof', 'anos', 'Inn', 'eny', 'adi', 'ritz', 'Pitts']

['benefits', 'rising', 'rapidly', 'quick', 'faster', 'massive', 'entially', 'evol', 'gent', 'atic', 'Rap', 'sudden', 'ancement', 'rapid', 'mediate', 'exponential', 'racing', 'rá']

['bow', 'rob', 'cc', 'elect', 'produ', 'ester', 'hosp', 'lad', 'igare', 'Rou', 'rou']

['Random', 'eth', 'Food', 'different', 'ately', 'String', 'first', 'element', 'remove', 'char', 'o

In [62]:
def getDict(loader):
    word_active_neurons = {}
    with torch.no_grad():
        for batch_num, batch in tqdm(enumerate(loader), total = len(loader), desc="Processing Batches"):
            # Batch to GPU
            batch, text = batch
            batch = batch.cuda().to(torch.float32)

            # Perform forward pass to get latent representations
            recons, auxk, num_dead, latents = model(batch)  # Assuming `model` returns latent as the second output

            # Find top K active neuron indices for each input in the batch
            K = 10  # Assuming TopK SAE is configured with K active neurons
            active_neuron_indices = torch.topk(latents, 128, dim=1).indices

            # Save results
            for i, indices in enumerate(active_neuron_indices):
                word = text[i]  # Assuming dataset has words or identifiers
                word_active_neurons[word] = indices.cpu().tolist()

            if batch_num % 100 == 0:
                print(f"Processed Batch {batch_num}...")
    return word_active_neurons

# # Save the mapping to a file
# with open("word_active_neurons.json", "w") as f:
#     json.dump(word_active_neurons, f)

# print("Forward pass completed and saved!")

In [64]:
tomSawyerDict = getDict(tomSawyer_loader)

Processing Batches: 100%|██████████| 2/2 [00:00<00:00,  6.89it/s]

Processed Batch 0...


In [65]:
prideAndPredDict = getDict(prideAndPred_loader)

Processing Batches: 100%|██████████| 2/2 [00:00<00:00,  8.07it/s]

Processed Batch 0...


In [66]:
from collections import Counter
def most_common_neurons(neuron_dict, K):
    # Flatten all indices into a single list
    all_indices = [index for indices in neuron_dict.values() for index in indices]
    
    # Count the frequency of each index
    index_counts = Counter(all_indices)
    
    # Sort by frequency (descending) and then by index (ascending)
    sorted_indices = sorted(index_counts.items(), key=lambda x: (-x[1], x[0]))
    
    # Return the top K indices
    return [index for index, _ in sorted_indices[:K]]

In [67]:

tomSawyerDict_themes = most_common_neurons(tomSawyerDict, 10)

In [73]:
tomSawyerDict_themes

[7409, 15362, 15271, 18441, 17306, 7587, 6103, 16430, 10140, 18326]

In [77]:
for i in tomSawyerDict_themes:
    print(len(get_word_list(i)))

13124
13126
13049
1509
12530
717
5571
4339
3433
385


In [68]:
prideAndPred_themes = most_common_neurons(prideAndPredDict, 10)

In [74]:
prideAndPred_themes

[7409, 15271, 15362, 17306, 18441, 7587, 6103, 23704, 16430, 3908]

In [76]:
for i in prideAndPred_themes:
    print(len(get_word_list(i)))

13124
13049
13126
12530
1509
717
5571
9828
4339
30


### For Tom Sawyer

In [80]:
print(get_word_list(18326))

['\n', 'pe', 'me', 'She', 'on', 'come', 'closer', 'I', 'she', 'found', 'proper', 'flav', 'still', 'immediately', 'will', 'ill', 'been', 'light', 'll', 'great', 'noticed', 'never', 'lines', 'thus', 'With', 'values', 'instead', 'he', 'something', 'home', 'live', 'y', 'even', 'itself', 'hur', 'escape', 'aim', 'Whether', 'try', 'filled', 'had', 'releases', 'secret', 'before', 'decided', 'Mark', 'bb', 'island', 'reduce', 'chance', 'builds', 'looks', 'im', 'achieve', 'journey', 'get', 'make', 'heard', 'amaz', 'dropped', 'susp', 'remain', 'reve', 'saf', 'seeking', 'tail', 'knows', 'enough', 'lived', 'ished', 'bring', 'discovered', 'edge', 'fun', 'ge', 'leaving', 'det', 'Tr', 'holding', 'bag', 'led', 'ilt', 'per', 'acles', 'further', 'remember', 'hill', 'figured', 'admitted', 'remembered', 'takes', 'heavy', 'Every', 'wonder', 'perfect', 'forces', 'happy', 'amount', 'probably', 'did', 'alone', '.”', 'secre', 'hard', 'den', 'ky', 'soft', 'deliver', 'reached', 'lied', 'wel', 'mission', 'hands', '

In [ ]:
# The overarching theme of this set of words appears to revolve around journeys, encounters, and emotional states

### For pride and Prejudice

In [81]:
print(get_word_list(3908))

['connected', 'King', 'German', 'couple', 'idas', 'hat', 'shall', 'Rome', 'acher', 'prince', 'Jul', 'iet', 'whom', 'marry', 'ome', 'tested', 'asks', 'honor', 'convinced', 'faithful', 'duty', 'commits', 'trag', 'recon', 'Father', 'cile', 'fore', 'thou', 'conflic', 'Honor']


In [82]:
# The theme of this collection centers on duty, honor, and interpersonal relationships, often framed in a historical or romantic context.

In [83]:
from collections import Counter
def most_common_neurons(neuron_dict, K):
    # Flatten all indices into a single list
    all_indices = [index for indices in neuron_dict.values() for index in indices]
    
    # Count the frequency of each index
    index_counts = Counter(all_indices)
    
    # Sort by frequency (descending) and then by index (ascending)
    sorted_indices = sorted(index_counts.items(), key=lambda x: (-x[1], x[0]))
    
    # Return the top K indices
    return [index for index, _ in sorted_indices[K:K+K]]

In [84]:
tomSawyerDict_themes = most_common_neurons(tomSawyerDict, 10)

In [85]:
tomSawyerDict_themes

[23704, 25660, 2948, 18538, 15144, 7788, 28030, 20709, 19313, 11838]

In [92]:
for i in tomSawyerDict_themes:
    print(f"Neuron Num, {i}, Len of its dict {len(get_word_list(i))}")

Neuron Num, 23704, Len of its dict 9828
Neuron Num, 25660, Len of its dict 7041
Neuron Num, 2948, Len of its dict 9199
Neuron Num, 18538, Len of its dict 147
Neuron Num, 15144, Len of its dict 2495
Neuron Num, 7788, Len of its dict 319
Neuron Num, 28030, Len of its dict 75
Neuron Num, 20709, Len of its dict 2645
Neuron Num, 19313, Len of its dict 985
Neuron Num, 11838, Len of its dict 2933


In [87]:
prideAndPred_themes = most_common_neurons(prideAndPredDict, 10)

In [88]:
prideAndPred_themes

[7788, 25660, 18538, 2948, 18326, 10140, 9179, 20709, 11838, 15144]

In [93]:
for i in prideAndPred_themes:
    print(f"Neuron Num, {i}, Len of its dict {len(get_word_list(i))}")

Neuron Num, 7788, Len of its dict 319
Neuron Num, 25660, Len of its dict 7041
Neuron Num, 18538, Len of its dict 147
Neuron Num, 2948, Len of its dict 9199
Neuron Num, 18326, Len of its dict 385
Neuron Num, 10140, Len of its dict 3433
Neuron Num, 9179, Len of its dict 59
Neuron Num, 20709, Len of its dict 2645
Neuron Num, 11838, Len of its dict 2933
Neuron Num, 15144, Len of its dict 2495


In [94]:
print(get_word_list(28030))

['cre', 'come', 'eth', 'ones', 'ians', 'kind', 'ever', 'instead', 'human', 'regime', 'live', 'even', 'secret', 'looks', 'heard', 'ole', 'neighbourhood', 'ank', 'river', 'judge', 'drink', 'bag', 'ilt', 'remember', 'hill', 'takes', 'ships', 'lack', '—', 'wel', 'mission', 'interrupt', 'itch', 'unn', 'strugg', 'ief', 'lif', 'males', 'killed', 'speaker', 'inspired', 'employee', 'resource', 'construct', 'shall', 'Rome', 'violent', 'asion', 'shell', 'icked', 'party', 'ric', 'acher', 'wid', 'prince', 'voyage', 'batt', 'elf', 'request', 'soup', 'white', 'idden', 'iet', 'compan', 'bomb', 'cons', 'whom', 'ury', 'itan', 'fails', 'marry', 'reb', 'returning', 'queen', 'Oregon', 'Upon', 'opening', 'brothers', 'conscience', 'Years', 'appeared', 'itution', 'brid', 'sheep', 'painted', 'gradually', 'Wolf', 'agger', 'originally', 'army', 'ego', 'boats', 'villa', 'convinced', 'elder', 'sib', 'faithful', 'anda', 'cres', 'wife', 'Mississippi', 'ruled', 'Monsieur', 'commits', 'forcing', 'uncle', 'tribes', 'La

In [ ]:
print(get_word_list(9179))

['Bob', 'Richard', 'cave', 'marriage', 'father', 'ocr', 'killed', 'says', 'Mar', 'China', 'ric', 'ier', 'wid', 'Mat', 'prince', 'loved', 'Ferdinand', 'eding', 'whom', 'cess', 'Franz', 'marry', 'Marcus', 'Mrs', 'Jac', 'Parliament', 'Cle', 'arrival', 'Haz', 'villa', 'Gar', 'Austria', 'etta', 'faithful', 'ixon', 'wife', 'cousin', 'relatives', 'Robinson', 'uw', 'uncle', 'ocket', 'Lan', 'assador', 'victim', 'mate', 'Buch', 'Conference', 'anie', 'Herbert', 'accus', 'slave', 'bian', 'Abd', 'adi', 'disapp', 'XVIII', 'nep', 'ouv']


In [113]:
def topK_activation(x, k):
    topk = torch.topk(x, k=k, dim=-1, sorted=False)
    values = F.relu(topk.values)
    result = torch.zeros_like(x)
    result.scatter_(-1, topk.indices, values)
    return topk



In [114]:
a = torch.randn((1, 900))


In [115]:
topK = topK_activation(a, 10)

In [116]:
topK.indices

tensor([[312, 335, 895, 683, 817,  28, 178,  26, 742, 124]])

In [117]:
result = torch.zeros_like(a)
result.scatter_(-1, topK.indices, a)

tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  1.4375,  0.0000,  0.8863,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  

In [71]:
sent = []
for ind in tomSawyerDict_themes:
    sent.append(' '.join(word for word in get_word_list(ind)))
    

In [96]:
# sent

In [22]:
print(get_word_list(129))  #Fractions

[]


In [23]:
print(get_word_list(30718)) # Leadership roles

[]


In [24]:
print(get_word_list(30717))

[]


In [25]:
print(get_word_list(30710))

[]


In [79]:
print(get_word_list(30709))

['game', 'played', 'Game', 'games', 'Games', 'sport', 'tery', 'Scene', 'Gam', 'amo']


In [ ]:
print(get_word_list(30704))